## BETTER WAY 38. 간단한 인터페이스의 경우 클래스 대신 함수를 받아라

파이썬 내장 API 상당수는 함수를 전달해서 동작을 원하는대로 바꿀 수 있게 해준다.
API가 실행되는 과정에서 여러분이 전달한 함수를 실행하는 경우, 이런 함수를 훅(hook)이라고 부른다.

ex) 리스트 타입의 sort 메서드는 정렬 시 각 인덱스에 대응하는 비교 값을 결정하는 선택적인 key인자를 받을 수 있다.

다음 코드는 key 훅으로 len 내장 함수를 전달해서 이름이 들어있는 리스트를 이름의 길이에 따라서 정렬한다.

In [1]:
names = ['소크라테스', '아르키메데스', '플라톤', '아리스토텔레스']
names.sort(key=len)
print(names)

['플라톤', '소크라테스', '아르키메데스', '아리스토텔레스']


훅을 추상 클래스(abstract class)를 통해 정의해야 하는 언어도 있지만, 파이썬에서는 단순히 인자와 변환 값이 잘 정의된, 상태가 없는 함수를 훅으로 사용하는 경우가 많다.

또한 파이썬은 함수를 **일급 시민 객체**로 취급하기 때문에 함수를 훅으로 사용할 수 있다.

- 일급 시민 객체 : 이는 함수를 변수에 할당하고, 함수를 인자로 다른 함수에 전달하고, 함수를 반환값으로 사용하는 것이 가능하다는 것을 의미

ex) defaultdict에는 딕셔너리 안에 없는 키에 접근할 경우 호출되는 인자가 없는 함수를 전달할 수 있다. 이 함수는 존재하지 않는 키에 해당하는 값이 될 객체를 반환해야 한다.

다음 코드는 존재하지 않는 키에 접근할 때 로그를 남기고 0을 디폴트 값으로 반환한다.

In [2]:
def log_missing():
    print('키 추가됨')
    return 0

In [4]:
# 원본 딕셔너리와 변경할 내용이 주어진 경우
# log_missing 함수는 로그를 두번 남길 수 있다.(각 로그는 red와 orange에 해당)

from collections import defaultdict

current = {'초록' : 12, '파랑' : 3}

increments = [
    ('빨강', 5),
    ('파랑', 17),
    ('주황', 9)
]

# 초기값이 있는 딕셔너리를 생성하는데, 누락된 값을 처리하기 위한 함수 log_missing과 초기값(current)를 인자로 넘겨줌
result = defaultdict(log_missing, current)
print('이전', dict(result))
# increments 리스트에 있는 각 튜플의 key와 amount를 이용해 defaultdict 객체의 값 업데이트
# 각 키에 대한 값이 없으면 log_missing 함수를 이용해 새로운 딕셔너리 항목 만듦
for key, amount in increments:
    result[key] += amount
print('이후', dict(result))

이전 {'초록': 12, '파랑': 3}
키 추가됨
키 추가됨
이후 {'초록': 12, '파랑': 20, '빨강': 5, '주황': 9}


log_missing과 같은 함수를 사용할 수 있으면 정해진 동작과 부수 효과를 분리할 수 있기 때문에 API를 더 쉽게 만들 수 있음

ex) defaultdict에 전달하는 디폴트 값이 훅이 존재하지 않는 키에 접근한 총 횟수를 세고 싶다고 하자.

-> 상태가 있는 클로저 사용

클로저(closure)
: 함수 내부에서 내부 함수를 정의하여, 이 내부 함수가 외부 함수의 지역 변수에 접근할 수 있게 만드는 기술.

이를 이용하면 함수 외부에서 지역 변수를 변경하는 것을 막으면서도, 내부 함수에서 이 값을 사용할 수 있다.


In [5]:
# 예시코드
def outer_func(x): # outer_fun는 인자 x를 받아
    def inner_func(y): # 내부 함수 inner_func를 정의한 뒤 이 함수를 반환
        return x + y 
    return inner_func

closure_func = outer_func(10) # outer_func(10)을 호출하여, closure_fun에 반환값 할당
result = closure_func(5) # closure_func는 내부함수 inner_func 참조, 이때 inner_func은 outer_fun의 인자 x에 접근가능
# 따라서 closure_fun(5)를 호출하면, x가 10이므로 15를 반환
print(result) # 15 출력

15


In [6]:
# 클로저가 있는 도우미 함수를 디폴트 값 훅으로 사용

def increment_with_report(current, increments):
    added_count = 0

    def missing():
        nonlocal added_count # increment_with_report의 지역변수를 참조하겠다.
        added_count += 1
        return 0
    
    result = defaultdict(missing, current)
    for key, amount in increments:
        result[key] += amount
    
    return result, added_count

인터페이스에서 간단한 함수를 인자로 받으면, 클로저 안에 상태를 감추는 기능 계층을 쉽게 추가할 수 있다.

In [8]:
result, count = increment_with_report(current, increments)
assert count == 2 # 주어진 조건이 False인 경우, 예외를 발생시키는 제어문

하지만 상태를 다루기 위한 훅으로 클로저를 사용하면 상태가 없는 함수에 비해 읽고 이해하기 어렵다.  

다른 접근 방법은 추적하고 싶은 상태를 저장하는 작은 클래스를 정의하는 것

In [9]:
class CountMissing:
    def __init__(self):
        self.added = 0
    
    def missing(self):
        self.added += 1
        return 0

파이썬에서 함수는 일급 객체 (first-class object) 이기 때문에 defaultdict의 디폴트 값 훅으로 CountMissing 클래스의 인스턴스를 직접 전달할 수 있다.  

CountMissing 클래스는 missing 메서드를 가지고 있으므로, 이 인스턴스를 전달하면 defaultdict이 이 메서드를 호출하여 누락된 값들을 처리할 수 있다.  

이처럼 파이썬에서는 함수를 일급 객체로 취급하기 때문에, 함수 인터페이스를 만족하는 객체를 만드는 것도 아주 쉽다.

In [11]:
counter = CountMissing()
result = defaultdict(counter.missing, current)
for key, amount in increments:
    result[key] += amount
assert counter.added == 2 

코드만 보면 increment_with_report 같은 함수를 사용하는 것 보다 깔끔.

하지만 클래스 자체만 놓고보면 CountMissing 클래스의 목적이 무엇인지 알기가 어려움.

Q1. 누가 CountMissing 객체를 만들까?  
Q2. 누가 missing 메서드를 호출할까?  
Q3. 나중에 공개 메서드가 더 추가될 수도 있을까?    
Defialtdict와 함께 사용하는 예제를 보기 전까지는 이 클래스는 잘 모름  

__call__로 더 명확히 표현 가능

**\_\_call__** : 해당 클래스의 인스턴스를 함수처럼 호출 가능하게 만드는 특수한 메서드
> \_\_call__ 메서드를 정의한 클래스의 인스턴스는 호출될 때 마다 '\_\_call__' 메서드가 실행됨

\_\_call__ 이 정의된 클래스의 인스턴스에 대해 callable 내장 함수를 호출하면 다른 일반 함수나 메서드와 마찬가지로 True가 반환된다.
> callable 객체 : 호출 가능한 객체를 의미. 즉, 함수와 메서드 같은 일반적인 객체뿐만 아니라, call 메서드가 정의되어 있는 객체도 callable 객체로 간주

In [14]:
class BetterCountMissing:
    def __init__(self):
        self.added = 0

    def __call__(self):
        self.added += 1
        return 0

counter = BetterCountMissing()
assert counter() == 0
assert callable(counter)

다음 코드는 BetterCountMissing 인스턴스를 defaultdict의 디폴트 값 훅으로 사용해서 존재하지 않는 키에 접근한 횟수를 추적한다.

In [15]:
counter = BetterCountMissing()
result = defaultdict(counter, current) #__call__에 의존함
for key, amount in increments:
    result[key] += amount

assert counter.added == 2

- CountMissing을 사용한 코드보다 훨씬 깔끔
- \_\_call__ 메서드는 함수가 인자로 쓰일 수 있는 부분에 이 클래스의 인스턴스를 사용할 수 있다는 사실을 나타냄
- 코드를 처음 읽는 사람도 이 클래스의 동작을 알아보기 위한 시작점이 \_\_call__임을 알 수 있음
- **가장 좋은 점** : defaultdict가 \_\_call__ 내부에서 어떤 일이 벌어지는지에 대해서 전혀 알 필요가 없다는 것 

**기억해야 할 내용**
- 파이썬의 여러 컴포넌트 사이에 간단한 인터페이스가 필요할 때는 클래스를 정의하고 인스턴스화 하는 대신 간단히 함수를 사용할 수 있다.

- 파이썬 함수나 메서드는 일급 시민이다. 따라서 다른 타입의 값과 마찬가지로 함수나 함수 참조를 식에 사용할 수 있다.

- \_\_call__ 특별 메서드를 사용하면 클래스의 인스턴스인 객체를 일반 파이썬 함수처럼 호출할 수 있다.

- 상태를 유지하기 위한 함수가 필요한 경우에는 상태가 있는 클로저를 정의하는 대신 \_\_call__ 메서드가 있는 클래스를 정의할지 고려해보라 